In [164]:
import pandas as pd
import numpy as np
import os

In [165]:
%pwd

'C:\\Users\\satka\\OneDrive\\Desktop\\recommendation-system\\data'

In [166]:
os.chdir("C:/Users/satka/OneDrive/Desktop/recommendation-system/data")

In [167]:
df = pd.read_csv("anime_data_24.csv")
df_links = pd.read_csv('anime_links.csv')

In [168]:
anime = df.merge(df_links,on='name')

In [169]:
anime = anime[['name','sypnopsis','image','type','episodes','status','studios','source','genres','demographic','links']]

In [170]:
anime = anime.iloc[0:8000]

In [171]:
anime.columns

Index(['name', 'sypnopsis', 'image', 'type', 'episodes', 'status', 'studios',
       'source', 'genres', 'demographic', 'links'],
      dtype='object')

In [172]:
anime.dropna(inplace=True)

In [173]:
anime['sypnopsis_length'] = [len(i) for i in anime.sypnopsis]

In [174]:
anime = anime[anime['sypnopsis_length'] > 300]

In [175]:
anime['tags'] = anime['sypnopsis']+" " + anime['type']+" " + anime['episodes']+" " + anime['status'] +" "+ anime['studios'] +" "+ anime['source']+" " + anime['genres']+" " + anime['demographic']

In [176]:
anime_1 = anime.copy()
anime_1 = anime_1[['image','name','tags','links']]

In [177]:
anime_1 = anime_1.reset_index(drop=True)

In [178]:
anime_1 = anime_1.rename({'name':'title'},axis=1)

In [179]:
anime_1.tags = anime_1.tags.map(lambda x: x.replace('\n'," "))

In [180]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,stop_words='english')

In [181]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [182]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [183]:
anime_1['tags'] = anime_1['tags'].apply(stem)

In [184]:
vectors = cv.fit_transform(anime_1['tags']).toarray()

In [185]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(anime):
    anime_index = anime_1[anime_1['title']== anime].index[0]
    distances = np.around(similarity[anime_index],2)
    anime_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:8]

    for i in anime_list:
        print(anime_1.iloc[i[0]].title)
        print(anime_1.iloc[i[0]].links)
        print(i[0])

In [203]:
recommend('Shingeki no Kyojin')

Shingeki no Kyojin
https://myanimelist.net/anime/16498/Shingeki_no_Kyojin
110
Shingeki no Kyojin Season 2
https://myanimelist.net/anime/25777/Shingeki_no_Kyojin_Season_2
123
Shingeki no Kyojin Season 3
https://myanimelist.net/anime/35760/Shingeki_no_Kyojin_Season_3
78
Shingeki no Kyojin: The Final Season Part 2
https://myanimelist.net/anime/48583/Shingeki_no_Kyojin__The_Final_Season_Part_2
42
Shingeki no Kyojin Season 2 Movie: Kakusei no Houkou
https://myanimelist.net/anime/36702/Shingeki_no_Kyojin_Season_2_Movie__Kakusei_no_Houkou
720
Shingeki no Kyojin: The Final Season - Kanketsu-hen
https://myanimelist.net/anime/51535/Shingeki_no_Kyojin__The_Final_Season_-_Kanketsu-hen
26
Tokyo Ghoul
https://myanimelist.net/anime/22319/Tokyo_Ghoul
935
Shingeki! Kyojin Chuugakkou
https://myanimelist.net/anime/31374/Shingeki_Kyojin_Chuugakkou
2606


In [190]:
sorted(list(enumerate(similarity[anime_1[anime_1['title']== 'Shingeki no Kyojin Season 3 Part 2'].index[0]])),reverse=True,key=lambda x:x[1])[1:6]

[(720, np.float64(0.46277514967945194)),
 (123, np.float64(0.39129428821566115)),
 (36, np.float64(0.32404377887774766)),
 (42, np.float64(0.3213720099233944)),
 (110, np.float64(0.31051721396879134))]

In [191]:
anime_1.iloc[720]

image    https://cdn.myanimelist.net/images/anime/1710/...
title    Shingeki no Kyojin Season 2 Movie: Kakusei no ...
tags     eren yeager and other of the 104th train corp ...
links    https://myanimelist.net/anime/36702/Shingeki_n...
Name: 720, dtype: object